Input: Hansen forest change  
https://github.com/Revalue-Nature/notebooks-early-assessment/blob/main/Global_Hansen_ForestChange_2023_v5.ipynb

### Import library

In [12]:
%matplotlib inline
import tkinter as ttk
from tkinter import Tk, filedialog, simpledialog, messagebox
import os
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.plot import show
from rasterio.warp import reproject, Resampling, calculate_default_transform
import numpy as np
from osgeo import gdal

### Specify input variables

In [13]:
# Specify input name 
jurisdictionname = "Sofala"
jrd_column_name = "NAME_1"
projection = "EPSG:32737"

In [14]:
# Specify time period
t1 = 2015
t2 = 2021

In [15]:
root = Tk() # pointing root to Tk() to use it as Tk() in program.
root.withdraw() # Hides small tkinter window.
root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
jrd_directory = filedialog.askopenfile(title="Open Jurisdiction Shapefile") # Returns opened path as st
# Raster folder
raster_directory = filedialog.askdirectory(title = "Specify Raster Folder Directory")

In [ ]:
# # Output folder
# output_directory = os.path.expanduser('./riskmapjnr/output')
# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)

In [16]:
# output directory
out_dir = os.path.expanduser('./riskmapjnr/data')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [17]:
# Jurisdiction
jrd = gpd.read_file(jrd_directory.name)
jrd_proj  = jrd.to_crs(crs=projection)

### Raster data preparation

In [18]:
# Import Hansen forest change raster
fcc_path = os.path.join(raster_directory, jurisdictionname + '_ForestChange_' + str(t1) + '-' + str(t2) + '_jrd.tif').replace(os.sep, '/')
fcc_tif = rasterio.open(fcc_path)

In [19]:
np.unique(fcc_tif.read(1))

array([  0, 100, 116, 117, 118, 119, 120, 121], dtype=uint8)

### FCC output

In [35]:
# get fcc123

# Reclassify raster pixel value
# 116-118 = 1
# 119-121 = 2
# 100 = 3

with rasterio.open(fcc_path) as src:    
    # Read as numpy array
    array = src.read()
    profile = src.profile
    profile.update(compress='lzw')

    # Reclassify
    array[((t1 - 2000 +100) < array) & (array <= (t1 - 2000 + 100 + 3))] = 1
    array[((t1 - 2000 + 100 + 3) < array) & (array <= (t2 - 2000 +100))] = 2
    array[array == 100] = 3
    # and so on ...  

with rasterio.open(os.path.join(out_dir, jurisdictionname + '_fcc123_' + str(t1+1) + '_' + str(t2) + '.tif').replace(os.sep, '/'), 'w', **profile) as dst:
    # Write to disk
    dst.write(array)

In [16]:
# get fcc23

# Reclassify raster pixel value
# 116-121 = 0
# 100 = 1

with rasterio.open(fcc_path) as src:    
    # Read as numpy array
    array = src.read()
    profile = src.profile
    profile.update(compress='lzw')

    # Reclassify
    array[((t1 - 2000 +100) < array) & (array <= (t2 - 2000 +100))] = 0 # loss t1 to t2
    array[array == 100] = 3 # forest remnant at the end of t2
    # and so on ...  

with rasterio.open(os.path.join(out_dir, jurisdictionname + '_fcc23_' + str(t1+1) + '_' + str(t2) + '.tif').replace(os.sep, '/'), 'w', **profile) as dst:
    # Write to disk
    dst.write(array)